In [3]:
import cv2
import os
import numpy as np
from random import randint
from shutil import copyfile
import csv

In [ ]:
# chia thu muc anh theo file 
train_fol = '/home/dev/aivivn/vn_celeb_face_recognition/train'
ftrain = open('/home/dev/aivivn/vn_celeb_face_recognition/train.csv', 'r')
catefol = '/home/dev/aivivn/catefol'
for x in ftrain:
    path, iid = x.split(',')
    if path.endswith('.png'):
        idfol = os.path.join(catefol, iid)
        print(idfol)
        if not os.path.isdir(idfol):
            print('Making new dir %s', idfol)
            os.mkdir(idfol)
        src = os.path.join(train_fol, path)
        dst = os.path.join(catefol, idfol, path)
        copyfile(src, dst)        

In [2]:
def isGray (img):
    w, h, c = img.shape
    for i in range(w):
        for j in range(h):
            pixel = (img[i, j])
            if pixel[0] != pixel [1] or pixel[0] != pixel [2] or pixel [1] != pixel[2]:
                print("rgb")
                return False
    print("gray")
    return True
   

In [ ]:
train_fol = '/home/dev/aivivn/1000classTrain'
all_train_root = '/home/dev/aivivn/traindataaug'

#  1: shift, 2: rot, 3: flip, 4: grayscale if rgb 
with open('traindataaugfile.csv' , 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                        quotechar='|', quoting=csv.QUOTE_NONE)
    writer.writerow(['image','label'])
    for root, dirs, files in os.walk(train_fol):
        for subdir in dirs:
            subfol = (os.path.join(train_fol, subdir))
            files = os.listdir(subfol)
            for file in files:
                if not file.endswith('.png'):
                    files.remove(file)
            for file in files:
#                 print(file)
                if file.endswith('.png'):
                    name = os.path.join(subfol, file)
                    img = cv2.imread(os.path.join(subfol, file))

                    if not isGray(img):
                        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        newname = 'gray' + file
                        cv2.imwrite(os.path.join(subfol, newname), gray)
            files = os.listdir(subfol)
            for file in files:
                if not file.endswith('.png'):
                    files.remove(file)
            numi = (len(files))
            print (numi)
    #         gen to 16 images for each class
            if numi < 16:
                for i in range( 16 - numi):
        #             choose image to process
                    imgid = randint(0, numi-1)
                    name = os.path.join(subfol, subdir + '_' + str(i) + '.png')
#                     print(os.path.join(subfol, files[imgid]))
                    img = cv2.imread(os.path.join(subfol, files[imgid]))
                    rows, cols, ch = img.shape

        #             choose method: shift, rot, flip
                    met = randint(1, 3)
                    if met == 1: # shift 
                        offsetx = randint(-20, 20)
                        offsety = randint(-20, 20)
                        M = np.float32([[1, 0, offsetx], [0, 1, offsety]])
                        dst = cv2.warpAffine(img, M, (cols, rows))
                        cv2.imwrite(name, dst)
                    if met == 2: 
                        rot = randint(-10, 10)
                        M = cv2.getRotationMatrix2D((cols/2, rows/2), rot, 1)
                        dst = cv2.warpAffine(img, M, (cols, rows))
                        cv2.imwrite(name, dst)
                    else:
                        dst = cv2.flip(img, 1)
                        cv2.imwrite(name, dst)
            newfiles = os.listdir(subfol)
#             print(len(newfiles))
            for newfile in newfiles:
                src = os.path.join(subfol, newfile)
                dst = os.path.join(all_train_root, newfile)
                copyfile(src, dst)
                writer.writerow([newfile, subdir])

In [ ]:
# gen file groundtruth cho new train image set 
